In [2]:
import numpy as np
import pandas as pd

seed = 3
np.random.seed(seed)

In [3]:
DATA_URL = "../../data_pipeline/notebooks/data/courses.csv"

courses_df = pd.read_csv(DATA_URL)
courses_df.head(1)

,CRN,Campus Description,Course Title,Subject Course,Faculty Name,Course Description,Term,Begin Time,End Time,Days,Prerequisites
0,34154,Online,Computer Science and Its Applications,CS1100,"Lieberherr, Karl",Introduces students to the field of computer s...,Spring 2025,NaN,NaN,NaN,[]


In [4]:
n_profs = 50

professors = courses_df["Faculty Name"].unique()
professors = np.random.choice(professors, size=n_profs, replace=False)

In [5]:
semester = "Spring 2025"

topics = [
    "Machine Learning",
    "Deep Learning",
    "Data Science",
    "Computer Systems",
    "Programming",
    "Development"
]

course_names = [
    'Foundations of Artificial Intelligence',
    'Algorithms',
    'Programming Design Paradigm',
    'Pattern Recognition and Computer Vision',
    'Natural Language Processing',
    'Deep Learning',
    'Machine Learning',
    'Information Retrieval',
    'Computer Systems',
    'Database Management',
    'Recitation for CS5010',
    'Foundations of Software Engineering',
    'Reinforcement Learning',
    'Computer/Human Interaction',
    'Web Development',
    'Compilers',
    'Mobile Application Development',
    'Discrete Structures',
    'Data Mining Techniques',
    'Fundamentals of Cloud Computing',
    'Object-Oriented Design',
    'Computer Systems'
]

In [6]:
seed_query_list = [
    "What courses are being offered for the {semester} semester?",
    "Which professor is taking the course {course_name}?",
    "What are the courses available related to {topic}?",
    "Can you please suggest online courses to take?",
    "Can you suggest research or seminar-based courses?",
    "Can you suggest courses without exams?",
    "Can you suggest courses which are not too hectic and easy to get good grades?",
    "What are the class timings for the course {course_name}?",
    "Are there any prerequisite courses for {course_name}?",
    "Has the course {course_name} been offered in the past? If so, could you summarize the reviews?",
    "How's the course {course_name} based on the reviews?",
    "How difficult is the course {course_name}?",
    "What courses are being offered by Professor {professor_name}?",
    "How is Professor {professor_name} in terms of grading?"
]

In [7]:
sft_dataset_creator_system_prompt = """
You are Qwen, a SFT Dataset creation tool. Given a question and context related to the question, you generate responses based on them.

Output Format:
Question:
{question}
Context:
{context}
Response:
{generated_response}
"""

prompt = "Context:\n{context}\n\nQuery: {query}\n\nAnswer:"

variant_system_prompt = """
You are Qwen, an LLM to help generate multiple variants of a question.

Given a question and the context, rephrase it in 10 different ways without altering it's meaning.
The rephrased question should have the same semantic meaning with different tone, style, etc.

Output Format:

The output should be in the following format.
{
    "question": [new_question_1, new_question_2, ..., new_question_10]
}
"""

In [8]:
sft_dataset = pd.DataFrame()

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [9]:
import torch
import gc

def free_gpu_memory():
    # Clear unused objects by forcing garbage collection
    gc.collect()

    # Empty the CUDA cache to release unused GPU memory
    torch.cuda.empty_cache()

    # Print current GPU memory usage for monitoring
    print(f"Current allocated GPU memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    print(f"Current cached GPU memory: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

free_gpu_memory()

Current allocated GPU memory: 0.00 MB
Current cached GPU memory: 0.00 MB


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def generate_response(system_prompt, user_prompt, model_name="Qwen/Qwen2.5-3B-Instruct"):
    # Load the model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Prepare the message and text for generation
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize the input and move to appropriate device
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Generate the output
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )

    # Decode and return the response
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [12]:
# from time import time

# system_prompt = "You are Qwen, the useful LLM"
# prompt = "Give me a short introduction to large language model."

# start_time = time()
# response = generate_response(system_prompt, prompt)
# print(response)
# end_time = time()

# print(f"Took {end_time - start_time} secs.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

A large language model (LLM) is a type of artificial intelligence that can generate human-like text based on patterns it has learned from vast amounts of text data. These models are trained using machine learning techniques, often employing deep neural networks, to understand and produce natural language in a way that mimics human language capabilities.

Key characteristics of large language models include:

1. **Size**: They are typically trained on massive datasets, which allows them to learn complex linguistic structures and context.
2. **Versatility**: They can perform a wide range of tasks, including text generation, translation, summarization, question-answering, and more.
3. **Adaptability**: They can adapt their outputs based on the input they receive, making them highly flexible for various applications.
4. **Contextual Understanding**: Modern large language models have been fine-tuned to understand and generate text with contextual awareness, meaning they can comprehend the b

In [23]:
# questions = []

# for seed_query in seed_query_list:
#     if "professor_name" in seed_query:
#         for professor_name in professors:
#             new_seed_query = seed_query.format(professor_name=professor_name)
            
#             # Make multiple variants of the same question
#             variant_response = generate_response(variant_system_prompt, new_seed_query)
#             variant_response = json.loads(variant_response)["question"]
#             questions.extend(variant_response)

#     elif "semester" in seed_query:
#         new_seed_query = seed_query.format(semester=semester)

#         # Make multiple variants of the same question
#         variant_response = generate_response(variant_system_prompt, new_seed_query)
#         variant_response = json.loads(variant_response)["question"]
#         questions.extend(variant_response)

#     elif "course_name" in seed_query:
#         for course_name in course_names:
#             new_seed_query = seed_query.format(course_name=course_name)

#             # Make multiple variants of the same question
#             variant_response = generate_response(variant_system_prompt, new_seed_query)
#             variant_response = json.loads(variant_response)["question"]
#             questions.extend(variant_response)

#     elif "topic" in seed_query:
#         for topic in topics:
#             new_seed_query = seed_query.format(topic=topic)

#             # Make multiple variants of the same question
#             variant_response = generate_response(variant_system_prompt, new_seed_query)
#             variant_response = json.loads(variant_response)["question"]
#             questions.extend(variant_response)

#     else:
#         # Make multiple variants of the same question
#         variant_response = generate_response(variant_system_prompt, seed_query)
#         variant_response = json.loads(variant_response)["question"]
#         questions.extend(variant_response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
# import pickle

# with open('questions.pkl', 'wb') as file:
#     pickle.dump(questions, file)

In [2]:
import pickle

with open("questions.pkl", "rb") as file:
    questions = pickle.load(file)

len(questions)

2430

In [ ]:
# TODO: Use the existing RAG Pipeline to fetch the context for the query

# TODO: Prompt an LLM to generate the response

# TODO: Save the question, context and response